# Final Challenge: The Code That Saved MAST

Agent, you've done it. You've overcome all seven challenges, outwitted Dr. Nefarious at every turn, and collected seven cryptic clues - each a single letter. It's time to put the pieces together and stop the deletion sequence once and for all.

---

## Imports and Setup

For this notebook, we'll import the following packages:
- `astroquery.mast` to access data from the MAST Archive.
- `astropy` to handle FITS.
- `lightcurve` for visualizing timeseries data.
- `mido` for sonifying timeseries data. This package is not included in TIKE's default environment, so we will have to install it with a magic command.

In [ ]:
!pip install mido

In [ ]:
from astroquery.mast import Observations, discovery_portal
from astropy.io import fits
import lightkurve as lk
from mido import MidiFile, MidiTrack, Message, MetaMessage

We'll also need to configure `astroquery.mast` to use MAST's test database.

In [ ]:
test_server = 'https://masttest.stsci.edu'
service_patch = '.24Test'

Observations._caom_all = Observations._caom_all + service_patch
Observations._caom_cone = Observations._caom_cone + service_patch
Observations._caom_filtered_position = 'Mast.Caom.Filtered' + service_patch + '.Position'
Observations._caom_filtered = Observations._caom_filtered + service_patch
Observations._caom_products = Observations._caom_products + service_patch

discovery_portal.PortalAPI.MAST_REQUEST_URL = test_server + "/api/v0/invoke"
discovery_portal.PortalAPI.COLUMNS_CONFIG_URL = test_server + "/portal/Mashup/Mashup.asmx/columnsconfig"
discovery_portal.PortalAPI.MAST_DOWNLOAD_URL = test_server + "/api/v0.1/Download/file"
discovery_portal.PortalAPI.MAST_BUNDLE_URL = test_server + "/api/v0.1/Download/bundle"

## Solve the Puzzle!

Each of the seven challenges you completed revealed a letter. When unscrambled, they spell a single word that is the key to unlocking MAST's hidden safeguard. Use the cell below to unscramble the clue letters and assign the code word to the variable `codeword`.

In [ ]:
codeword = ''  # Enter the code word here!

Hmmm, how peculiar! What could this code word mean?

Maybe we should try searching for it in the MAST Archive. Let's see if we can find any datasets related to this code word using the `Observations.query_criteria` method. We'll search for observations that have a target name matching the code word.

In [ ]:
secret_obs = Observations.query_criteria(target_name=codeword)
secret_obs

Look at that! The query returned a single observation that looks quite... interesting. Let's fetch its data products and take a closer look.

In [ ]:
secret_prod = Observations.get_product_list(secret_obs)
secret_prod

The observation returned a single timeseries data product. Let's download the file and examine it a bit more.

In [ ]:
manifest = Observations.download_products(secret_prod)

In [ ]:
# Open the product and read the data
filepath = manifest[0]['Local Path']
with fits.open(filepath) as hdul:
    hdul.info()
    time = hdul[1].data['TIME']
    flux = hdul[1].data['PDCSAP_FLUX'].astype(int)

Looks like a pretty standard light curve file. Let's use the `lightkurve` package to visualize the timeries data. We'll plot the flux against time to see if there's anything unusual.

In [ ]:
lc = lk.TessLightCurveFile(filepath)
lc.plot()

Wow, that's quite a light curve! It doesn't look like anything we've seen before, and it seems to have a... refrain?

Dr. Nefarious is a lover of fine music, so let's visualize this light curve in a different way. We'll sonify the timeseries data using the `mido` package to create a MIDI file that plays the light curve as music.

In [ ]:
# Create a new MIDI file and track
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

# Program = 65 (Alto Saxophone)
track.append(Message('program_change', program=65, time=0))

# Define the tempo (BPM) and calculate the microseconds per beat
bpm = 76
microseconds_per_beat = int(60_000_000 / bpm)
ticks_per_beat = mid.ticks_per_beat

# Set the tempo (this applies to the whole track)
track.append(MetaMessage('set_tempo', tempo=microseconds_per_beat))

# Set note duration (based on time difference)
for i in range(len(flux) - 1):
    pitch = flux[i]  # Get the pitch for the note
    duration = time[i + 1] - time[i]  # Time difference between successive points
    ticks = int(duration * ticks_per_beat)  # Convert to ticks

    # Add 'note_on' message to start the note
    track.append(Message('note_on', note=pitch, velocity=100, time=0))

    # Add 'note_off' message to stop the note after the calculated duration
    track.append(Message('note_off', note=pitch, velocity=100, time=ticks))

# Save the MIDI file
mid.save("super_secret_lightcurve.mid")

Once you've created the MIDI file, you can download it by right clicking on the file in the file browser and selecting "Download". Then, you can open the file with your favorite MIDI player to hear the light curve as music. You might recognize its melody!

## Epilogue: A Song in the Stars

As the light curve's song plays out, the deletion countdown halts.
Dr. Nefarious, watching from his orbital data fortress, is stunned.

*“What’s this? The Archive… it sings?”*

Moved by the unexpected beauty encoded in the starlight, Dr. Nefarious hesitates. He listens. A moment passes — then another.
And finally… he smiles.

*“Very well,” he says. “I suppose some things are worth preserving.”*

The danger is gone. The data is safe. And the stars — they whisper on.

**Mission accomplished, agent. Welcome to the next generation of MAST users.**

## About this Notebook

**Author:** Sam Bianco <br>
**Keywords:** AAS 246, Sonification, MAST Data Challenge <br>
**Last Updated:** June 2025 <br>
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 